In [7]:
import numpy as np
import cv2
from mango_classifier import NaiveMangoClassifier  # assuming you've named it this way
from helper import crop_mango_direct, crop_mango, resize_and_pad, MODEL
from collections import deque, Counter

import contextlib

In [10]:
# Initialize the mango classifier
classifier = NaiveMangoClassifier()  # replace with your actual path or method
classifier.train('datasets/mango_cut/images/train/ripe', 'datasets/mango_cut/images/train/raw')

# Load your input (either image or video)
input_source = "perfectmango.mp4"

# This will hold the last 20 predictions
last_20_preds = deque(maxlen=20)

CROP_CONFIDENCE = 0.4  # The confidence threshold for the mango detection

def most_common_prediction(predictions):
    # Return the most common prediction, or the last prediction if there's no detection
    count = Counter(predictions)
    if len(count) == 0:
        return None
    return count.most_common(1)[0][0]

# If it's a video, we'll iterate over frames, otherwise, just process the single image
if input_source.endswith('.mp4') or input_source.endswith('.avi') or input_source.endswith('.gif'):
    cap = cv2.VideoCapture(input_source)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('processed_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame to a temporary file, then process it
        cv2.imwrite("temp_frame.jpg", frame)
        cropped_img, result_flag = crop_mango(MODEL, "temp_frame.jpg", CROP_CONFIDENCE)

        if result_flag == 0:
            final_img = resize_and_pad(cropped_img)
            prediction = classifier.predict_direct(final_img)
            last_20_preds.append(prediction)  # Add the latest prediction to the last 20

            # Calculate the most common prediction from the last 20 frames
            display_prediction = most_common_prediction(last_20_preds)

            # Add the result text to the frame
            cv2.putText(frame, f"Prediction: {display_prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            # If no mango detected, simply display the last "raw" or "ripe" prediction, or "No mango detected" if there are no previous predictions.
            if most_common_prediction(last_20_preds):
                cv2.putText(frame, f"Prediction: {most_common_prediction(last_20_preds)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "No mango detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write the processed frame to the output video
        out.write(frame)

    # Close the video files
    cap.release()
    out.release()

else:
    cropped_img, result_flag = crop_mango_direct(MODEL, input_source, CROP_CONFIDENCE)

    if result_flag == 0:
        final_img = resize_and_pad(cropped_img)
        prediction = classifier.predict_direct(final_img)

        # Add the result text to the image
        cv2.putText(final_img, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imwrite('processed_image.jpg', final_img)
    else:
        print("No mango detected in the image!")

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 640x384 1 person, 1 orange, 1 potted plant, 71.1ms
Speed: 2.2ms preprocess, 71.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 orange, 1 potted plant, 48.9ms
Speed: 1.4ms preprocess, 48.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 orange, 1 potted plant, 49.2ms
Speed: 1.4ms preprocess, 49.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 orange, 1 potted plant, 60.9ms
Speed: 1.4ms preprocess, 60.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 orange, 1 potted plant, 68.4ms
Speed: 1.8ms preprocess, 68.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 orange, 1 potted plant, 56.2ms
Speed: 1.7ms prepro

Skipping due to low confidence.
Skipping due to low confidence.


0: 640x384 2 apples, 50.1ms
Speed: 1.6ms preprocess, 50.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 apples, 46.9ms
Speed: 1.5ms preprocess, 46.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 apples, 51.0ms
Speed: 1.8ms preprocess, 51.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 apples, 48.0ms
Speed: 1.5ms preprocess, 48.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 apples, 1 orange, 48.4ms
Speed: 1.5ms preprocess, 48.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)


Skipping due to low confidence.
Skipping due to low confidence.
Skipping due to low confidence.
Skipping due to low confidence.



0: 640x384 1 person, 2 apples, 50.1ms
Speed: 1.5ms preprocess, 50.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 2 apples, 48.4ms
Speed: 1.7ms preprocess, 48.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 apple, 49.0ms
Speed: 1.5ms preprocess, 49.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 apple, 48.2ms
Speed: 1.6ms preprocess, 48.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)


...

Skipping due to no detection.
Skipping due to no detection.
Skipping due to no detection.


0: 640x384 (no detections), 57.2ms
Speed: 1.7ms preprocess, 57.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 60.4ms
Speed: 2.2ms preprocess, 60.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 61.5ms
Speed: 1.6ms preprocess, 61.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)



Skipping due to no detection.
Skipping due to no detection.
Skipping due to no detection.


0: 640x384 (no detections), 57.1ms
Speed: 1.5ms preprocess, 57.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 77.1ms
Speed: 1.8ms preprocess, 77.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)


Skipping due to no detection.
Skipping due to no detection.
